## Import data

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from utils import *
import matplotlib.pyplot as plt
import matplotlib

data_path = '../data/SNL_MSU_DOE_raw.xlsx'

dfs = pd.read_excel(
    data_path, engine='openpyxl', sheet_name=None)

exclude_sheet = ['Resins', 'Fabrics','Environmental','Recent misc.']

sheet_names = list(dfs.keys())

for ex in exclude_sheet:
    if ex in sheet_names:
        sheet_names.pop(sheet_names.index(ex))

name_mapping = {
    'Material': 'Material',
    'Resin Type': 'Resin Type',
    'Vf, %': 'Fibre Volumn Fraction',
    '%, 0 Deg': 'Percentage of Fibre in 0-deg Direction',
    '%, 45 Deg': 'Percentage of Fibre in 45-deg Direction',
    '%, 90 Deg': 'Percentage of Fibre in 90-deg Direction',
    'other %': 'Percentage of Fibre in Other Direction',
    'Thickness, mm': 'Thickness',
    'Max. Stress, MPa': 'Maximum Stress',
    'Min. Stress, MPa': 'Minimum Stress',
    'R-value': 'R-value',
    'Freq., Hz': 'Frequency',
    'E, GPa': 'Initial Elastic Modulus',
    'Max. % Strain': 'Maximum Strain',
    'Min. % Strain': 'Minimum Strain',
    'Cycles': 'Cycles to Failure',
    'Moisture Gain, %': 'Moisture Gain',
    'Testing Temperature, OC': 'Temperature',
    'Width, mm': 'Width',
    'Static Max. Stress, MPa': 'Static Maximum Tensile Stress',
    'Static Min. Stress, MPa': 'Static Maximum Compressive Stress',
    'Static E, GPa': 'Static Elastic Modulus',
    'Static Max. % Strain': 'Static Maximum Tensile Strain',
    'Static Min. % Strain': 'Static Maximum Compressive Strain',
}


In [2]:
df_all = pd.concat([dfs[name] for name in sheet_names], axis=0, ignore_index=True)

plot_absence(df_all, name_mapping, '../data/SNL_MSU_DOE_absence_ratio_initial.png', fontsize=12)

## Preprocess

### Fix missing and useless cells

In [3]:
df_all = pd.concat([dfs[name] for name in sheet_names], axis=0, ignore_index=True)

modify_col(df_all, 'Testing Temperature, OC', remove_s, s=' ̊C')
modify_col(df_all, '%, 45 Deg', remove_s, s=' G')
modify_col(df_all, '%, 0 Deg', remove_s, s=' C')
modify_col(df_all, 'other %', remove_s, s=' G')
modify_col(df_all, 'Vf, %', cal_fraction, s='/')
modify_col(df_all, '%, 0 Deg', cal_fraction, s='-')
modify_col(df_all, '%, 45 Deg', cal_fraction, s='-')
modify_col(df_all, 'Thickness, mm', conditional_remove, s='mm dia') #
modify_col(df_all, 'Thickness, mm', conditional_remove, s='/') #
modify_col(df_all, 'Max. Stress, MPa', remove_s, s='*')
modify_col(df_all, 'Max. Stress, MPa', remove_s, s='+')
modify_col(df_all, 'Max. Stress, MPa', conditional_remove, s='Newtons')
modify_col(df_all, 'Min. Stress, MPa', conditional_remove, s='v')
modify_col(df_all, 'R-value', conditional_remove, s='*')
modify_col(df_all, 'R-value', conditional_replace, s1='static compression', s2='static')
modify_col(df_all, 'Max. % Strain', conditional_remove, s='----')
modify_col(df_all, 'Max. % Strain', remove_s, s='+')
modify_col(df_all, 'Min. % Strain', conditional_remove, s='Runout')
modify_col(df_all, 'E, GPa (0.1-0.3%)', conditional_remove, s='----')
modify_col(df_all, 'E, GPa', remove_strs)
modify_col(df_all, 'Initial cracking strain, %', remove_strs)
modify_col(df_all, 'Runout', str2num, s='Runout',n=1)
modify_col(df_all, 'Runout', str2num, s='runout',n=1)
# modify_col(df_all, 'Testing Temperature, OC', fill_na, n=20)

# These columns use NaN to represent absence of fibre in the direction, so simply fillna by 0.
fill_na_col = ['%, 0 Deg', '%, 45 Deg','%, 90 Deg','other %','45 Deg fabric','90 deg fabric','Runout']
tmp = df_all[fill_na_col].fillna(0)
df_all.loc[:,fill_na_col] = tmp

def merge_col(df, from_col, to_col):
    if from_col in list(df.columns) and to_col in list(df.columns):
        where = np.where(df[from_col].notna())[0]
        df.loc[where, to_col] = df.loc[where, from_col]
        del df[from_col]

# These columns have different names in different sheets
merge_col(df_all, 'E, GPa (0.1-0.3%)', 'E, GPa')
merge_col(df_all, 'Freq., Hz or mm/s', 'Freq., Hz')

# Some static experiments do not have R-value and cycles to failure are 1.
df_all.loc[np.where(df_all['Cycles'] == 1)[0], 'R-value'] = 'static'

### Calculate missing Max/Min stress data using R-value

In [4]:
# If R>1, max stress must be negative
where_R1 = []
for idx,x in enumerate(df_all['R-value']):
    if not isinstance(x,str) and x>1:
        where_R1.append(idx)
where_R1 = np.array(where_R1)
df_all.loc[where_R1,'Max. Stress, MPa'] = -np.abs(df_all.loc[where_R1,'Max. Stress, MPa'])

In [5]:
static_indexes = np.where(df_all['R-value']=='static')[0]
non_static_indexes = np.setdiff1d(df_all.index, static_indexes)
miss_max_stress_indexes = np.where(np.isnan(df_all['Max. Stress, MPa']))[0]
miss_min_stress_indexes = np.where(np.isnan(df_all['Min. Stress, MPa']))[0]
miss_R_value_indexes = np.where(pd.isna(df_all['R-value']))[0]

for idx in miss_max_stress_indexes:
    if idx in non_static_indexes and type(df_all.loc[idx,'R-value'])!=str:
        df_all.loc[idx,'Max. Stress, MPa'] = df_all.loc[idx,'Min. Stress, MPa']/df_all.loc[idx,'R-value']

for idx in miss_min_stress_indexes:
    if idx in non_static_indexes and type(df_all.loc[idx,'R-value'])!=str:
        df_all.loc[idx,'Min. Stress, MPa'] = df_all.loc[idx,'Max. Stress, MPa']*df_all.loc[idx,'R-value']

for idx in miss_R_value_indexes:
    if idx in non_static_indexes:
        df_all.loc[idx,'R-value'] = df_all.loc[idx,'Min. Stress, MPa']/df_all.loc[idx,'Max. Stress, MPa']

In [6]:
# Add material code
material_code = [str(df_all.loc[x, 'Material']) + str(df_all.loc[x, 'Lay-up']) for x in range(len(df_all))]
df_all['Material_Code'] = material_code

In [7]:
df_all.to_excel('../data/SNL_MSU_DOE_combine.xlsx', engine='openpyxl', index=False)

## Translate laminate code

In [8]:
if 'Lay-up' in df_all.columns:
    code2seq_dict = {}
    layups = df_all['Lay-up'].values
    for layer in list(set(layups)):
        code2seq_dict[layer] = code2seq(layer)
    seq = []
    for layer in layups:
        seq.append('/'.join([str(x) for x in code2seq(layer)]))
    df_all['Sequence'] = seq

### Extract and save static and fatigue data respectively

In [9]:

# hide_cols = ['Lay-up','Resin Type','Resin','0 Deg fabric','45 Deg fabric','90 deg fabric','Cure / Post Cure','Process','Test #','Coupon']

df_tmp = df_all.copy()
# for col in hide_cols:
#     del df_tmp[col]

df_static = df_tmp.loc[static_indexes].copy()
df_fatigue = df_tmp.loc[non_static_indexes].copy()

df_static.reset_index(drop=True, inplace=True)
df_fatigue.reset_index(drop=True, inplace=True)

modify_col(df_fatigue, 'Freq., Hz', conditional_remove, s='mm/s')

# df_fatigue.to_excel('../data/SNL_MSU_DOE_fatigue.xlsx', engine='openpyxl', index=False)
df_static['log(Static Maximum Tensile Stress)'] = np.log10(df_static['Max. Stress, MPa'].values.astype(float))
replace_column_name(df_static, name_mapping).to_excel('../data/SNL_MSU_DOE_static.xlsx', engine='openpyxl', index=False)

/home/xlluo/anaconda3/envs/mlfatigue/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in log10
  


### Extract material properties from static experiments

In [10]:
static_mat_lay = df_static['Material_Code'].copy()
static_properties = {}

static_features = ['Max. Stress, MPa', 'Min. Stress, MPa', 'E, GPa', 'Max. % Strain', 'Min. % Strain']

for material in list(set(static_mat_lay)):
    where_material = np.where(static_mat_lay == material)[0]
    # print(material, len(where_material))
    material_data = df_static.loc[where_material, static_features].copy()
    material_data.reset_index(drop=True, inplace=True)
    material_df = {}
    for feature in static_features:
        for idx in range(len(material_data[feature])):
            if type(material_data.loc[idx, feature]) == str:
                material_data.loc[idx, feature] = np.nan

        presence_indexes = np.where(material_data[feature])[0]
        mean_value = np.mean(material_data.loc[presence_indexes, feature])
        material_df[feature] = mean_value

    material_df = pd.DataFrame(material_df, index=[0])
    static_properties[material]=material_df

fatigue_static_features = ['Static '+x for x in static_features]

fatigue_mat_lay = df_fatigue['Material_Code'].copy()

df_fatigue[fatigue_static_features] = np.nan

for material in list(set(static_mat_lay)):
    where_material = np.where(fatigue_mat_lay == material)[0]
    if len(where_material) > 0:
        static_property = static_properties[material]
        for feature in static_features:
            df_fatigue.loc[where_material,'Static '+feature] = static_property[feature].values[0]


In [11]:
# Remove runout tests
df_fatigue = df_fatigue.loc[np.where(df_fatigue['Runout'] == 0)[0],:].copy()
df_fatigue.reset_index(drop=True, inplace=True)

In [12]:
df_fatigue.loc[np.where(df_fatigue['Static Max. Stress, MPa']<0)[0],'Static Max. Stress, MPa'] = np.nan
df_fatigue.loc[np.where(df_fatigue['Static Min. Stress, MPa']>0)[0],'Static Min. Stress, MPa'] = np.nan
df_fatigue.loc[np.where(df_fatigue['Static Max. % Strain']<0)[0],'Static Max. % Strain'] = np.nan
df_fatigue.loc[np.where(df_fatigue['Static Min. % Strain']>0)[0],'Static Min. % Strain'] = np.nan

df_fatigue = replace_column_name(df_fatigue, name_mapping)

In [13]:
df_fatigue['log(Cycles to Failure)'] = np.log10(df_fatigue['Cycles to Failure'].values.astype(float))

## Averaging

In [14]:
df_static = replace_column_name(df_static, name_mapping)
averaging(df_fatigue, measure_features=['Maximum Stress', 'Minimum Stress', 'Frequency']).to_excel('../data/SNL_MSU_DOE_avg_fatigue.xlsx', engine='openpyxl', index=False)
averaging(df_static, measure_features=['Cycles to Failure']).to_excel('../data/SNL_MSU_DOE_avg_static.xlsx', engine='openpyxl', index=False)

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

In [15]:
df_fatigue.to_excel('../data/SNL_MSU_DOE_fatigue.xlsx', engine='openpyxl', index=False)

### Calculate and plot absence ratios

In [16]:
plot_absence(df_fatigue, name_mapping, '../data/SNL_MSU_DOE_absence_ratio.png', fontsize=12)

In [17]:
plot_absence(df_static, name_mapping, '../data/SNL_MSU_DOE_static_absence_ratio.png', fontsize=12)